In [1]:
import osrm
import shapely.wkt
import geojson
from requests import get, post
from pandas import json_normalize
from geopy import distance
from shapely.wkt import loads
import math
import json


osrm.RequestConfig
distance.GeodesicDistance.ELLIPSOID = 'WGS-84'
d = distance.distance

In [2]:
def get_elevation(lat = None, long = None):
    '''
        script for returning elevation in m from lat, long
    '''
    if lat is None or long is None: return None
    
    query = ('http://localhost:9967/api/v1/lookup'
             f'?locations={str(lat)},{str(long)}')
    
    # Request with a timeout for slow responses
    r = get(query, timeout = 20)

    # Only get the json response in case of 200 or 201
    if r.status_code == 200 or r.status_code == 201:
        elevation = json_normalize(r.json(), 'results')['elevation'].values[0]
    else: 
        elevation = None
    return elevation

def get_bulk_elevation(payload):
    query = 'http://localhost:9967/api/v1/lookup'
    r = post(query, json=payload, timeout = 20)
    # Only get the json response in case of 200 or 201
    if r.status_code == 200 or r.status_code == 201:
        elevation = r.json()
    else: 
        elevation = None
    return elevation

class CO2Engine():
    class CO2Profile():
        def __init__(self):
            self.mass_vehicle = 32000 # in kilograms
            self.mass_load = 10000 # in kilograms
            self.accel_gravity = 9.8 # m/s^2
            self.accel_vehicle = 1 # m/s^2
            self.wheel_radius = 1
            self.radius_tyre = 0.515 # 515mm Tyre radius: FMX84 
            self.torque_engine_max = 2600 # Nm, ATO2612 gearbox output, D11 380 hp (279 kW)
            self.load_nominal = 0.6
            self.milage_per_liter = 8.4746 #kmpl, community average with 11.8km per 100km, assumed at 60% load (load_nominal)
            self.mileage_modifer = 4 #the vehicle can go plus or minus modifer amount
            self.displacement_d13 = 12.8 # dm^3 or litre, D13 variant
            self.axle_ratio_highest = 4.11 #maximum torque conversion at rear for power delivery, RSS1370A
            self.emission_co2 = 3.17 # g per g of diesel
            self.diesel_l_to_g = 850.8 #g per l
            pass
    
    def __init__(self, co2Profile: CO2Profile) -> None:
        self.co2profile = co2Profile
        self.distanceAccumulated_m = 0
        self.co2Accumulated_g = 0
        self.fuelAccumulated_l = 0
        pass

    def GetFuelEmissionsCharacteristics(self, distance_m, theta_deg):
        mass_total = self.co2profile.mass_vehicle + self.co2profile.mass_load
        force_engine = mass_total * self.co2profile.accel_vehicle
        force_weight = mass_total * self.co2profile.accel_gravity
        force_angular = force_weight * math.sin(math.radians(theta_deg))
        force_engine_max = (self.co2profile.torque_engine_max * self.co2profile.axle_ratio_highest) / self.co2profile.radius_tyre
        load = force_angular / force_engine_max
        load_offset = load - self.co2profile.load_nominal
        milage_compensated = (self.co2profile.milage_per_liter - ((load_offset) * self.co2profile.mileage_modifer))
        fuel_consumed = ((distance_m * 0.001) / milage_compensated)
        co2_consumption = self.co2profile.emission_co2 * (fuel_consumed * self.co2profile.diesel_l_to_g)
        return {"co2_consumption_g": co2_consumption, "fuel_consumed_l": fuel_consumed}
    
    def AccumulateFuelConsumption(self, distance_m, theta_deg):
        self.distanceAccumulated_m += distance_m
        co2Characteristics = self.GetFuelEmissionsCharacteristics(distance_m=distance_m, theta_deg=theta_deg)
        self.co2Accumulated_g += co2Characteristics['co2_consumption_g']
        self.fuelAccumulated_l += co2Characteristics['fuel_consumed_l']
        return co2Characteristics

        


In [8]:
g2

{"geometry": {"coordinates": [[11.57297, 48.17806], [11.57294, 48.17806], [11.57274, 48.17804], [11.57257, 48.17801], [11.57238, 48.17799], [11.57213, 48.17795], [11.57189, 48.1779], [11.57175, 48.17786], [11.5715, 48.17781], [11.57127, 48.17774], [11.57107, 48.17768], [11.57096, 48.17764], [11.57092, 48.17762], [11.57079, 48.17758], [11.57066, 48.17755], [11.57039, 48.17748], [11.56999, 48.17737], [11.56986, 48.17733], [11.56927, 48.17716], [11.56915, 48.17712], [11.56887, 48.17705], [11.5684, 48.17694], [11.56788, 48.17679], [11.56779, 48.17677], [11.56769, 48.17676], [11.56754, 48.17672], [11.5675, 48.17649], [11.56749, 48.17639], [11.56744, 48.17623], [11.56736, 48.17613], [11.56728, 48.17606], [11.56713, 48.17598], [11.56702, 48.17595], [11.56681, 48.17588], [11.56661, 48.17581], [11.56656, 48.17579], [11.56646, 48.17574], [11.56644, 48.17573], [11.56636, 48.17568], [11.56627, 48.1756], [11.56621, 48.17556], [11.56616, 48.17548], [11.56614, 48.17545], [11.56607, 48.1753], [11.5659

In [7]:
route = osrm.simple_route([11.572970151901245,48.178062914051345], [11.5685498714447,48.16769546012316],
                      output='route', overview="full", geometry='wkt', alternatives=True, steps=True)
g1 = shapely.wkt.loads(route[1]['geometry'])
g2 = geojson.Feature(geometry=g1, properties={})
locations_payload = []
path_db={}
engine = CO2Engine(CO2Engine.CO2Profile())

distance = -1
previousGeometry = ()
for val in g2.geometry['coordinates']:
    val_payload = {"latitude": val[1], "longitude": val[0]}
    if not val[1] in path_db:
        path_db[val[1]] = {}
    if not val[0] in path_db[val[1]]:
        path_db[val[1]][val[0]] = {}
    if distance == -1:
        path_db[val[1]][val[0]]['refPoint'] = None
        path_db[val[1]][val[0]]['distance'] = 0
        distance = 0
        previousGeometry = (val[1],val[0])
    else:
        delta_distance = d(previousGeometry, (val[1],val[0])).meters
        path_db[val[1]][val[0]]['refPoint'] = previousGeometry
        path_db[val[1]][val[0]]['distance'] = delta_distance
        distance += delta_distance
        previousGeometry = (val[1],val[0])
    locations_payload.append(val_payload)
elevation_payload = {"locations": locations_payload}
dataPayload = json.dumps(elevation_payload)
elevation_results = get_bulk_elevation(elevation_payload)
for val in elevation_results['results']:
    path_db[val['latitude']][val['longitude']]['elevation'] = val['elevation']
unwrapped_pathdb = {}
for lat in path_db:
    for long in path_db[lat]:
        unwrapped_pathdb[(lat,long)] = path_db[lat][long]
cumulative_altitude_change = -1
previousElevation = 0
for item in unwrapped_pathdb:
    if cumulative_altitude_change == -1:
        cumulative_altitude_change = unwrapped_pathdb[item]['elevation']
        previousElevation = unwrapped_pathdb[item]['elevation']
        unwrapped_pathdb[item]['angleChange'] = 0
        co2char = engine.AccumulateFuelConsumption(unwrapped_pathdb[item]['distance'], unwrapped_pathdb[item]['angleChange'])
        unwrapped_pathdb[item]['co2Delta_g'] = co2char['co2_consumption_g']
        unwrapped_pathdb[item]['fuelUsageDelta_l'] = co2char['fuel_consumed_l']
        
    else:
        delta_altitude = previousElevation - unwrapped_pathdb[item]['elevation']
        cumulative_altitude_change += delta_altitude
        unwrapped_pathdb[item]['angleChange'] = math.degrees(math.atan(delta_altitude / unwrapped_pathdb[item]['distance']))
        previousElevation = unwrapped_pathdb[item]['elevation']
        co2char = engine.AccumulateFuelConsumption(unwrapped_pathdb[item]['distance'], unwrapped_pathdb[item]['angleChange'])
        unwrapped_pathdb[item]['co2Delta_g'] = co2char['co2_consumption_g']
        unwrapped_pathdb[item]['fuelUsageDelta_l'] = co2char['fuel_consumed_l']
unwrapped_pathdb

{(48.17806, 11.57297): {'refPoint': None,
  'distance': 0,
  'elevation': 507,
  'angleChange': 0,
  'co2Delta_g': 0.0,
  'fuelUsageDelta_l': 0.0},
 (48.17806, 11.57294): {'refPoint': (48.17806, 11.57297),
  'distance': 2.23104591287274,
  'elevation': 507,
  'angleChange': 0.0,
  'co2Delta_g': 0.5533271241857763,
  'fuelUsageDelta_l': 0.00020516119331954647},
 (48.17804, 11.57274): {'refPoint': (48.17806, 11.57294),
  'distance': 15.038977981787172,
  'elevation': 508,
  'angleChange': -3.804218590127825,
  'co2Delta_g': 2.5132038662344596,
  'fuelUsageDelta_l': 0.0009318391991187583},
 (48.17801, 11.57257): {'refPoint': (48.17804, 11.57274),
  'distance': 13.075283557130126,
  'elevation': 508,
  'angleChange': 0.0,
  'co2Delta_g': 3.242832882477333,
  'fuelUsageDelta_l': 0.0012023691498657537},
 (48.17799, 11.57238): {'refPoint': (48.17801, 11.57257),
  'distance': 14.303908082908608,
  'elevation': 508,
  'angleChange': 0.0,
  'co2Delta_g': 3.5475470399182956,
  'fuelUsageDelta_l':

In [126]:
unwrapped_pathdb.keys()

dict_keys([(48.17806, 11.57297), (48.17806, 11.57294), (48.17804, 11.57274), (48.17801, 11.57257), (48.17799, 11.57238), (48.17795, 11.57213), (48.1779, 11.57189), (48.17786, 11.57175), (48.17781, 11.5715), (48.17774, 11.57127), (48.17768, 11.57107), (48.17764, 11.57096), (48.17762, 11.57092), (48.17758, 11.57079), (48.17755, 11.57066), (48.17748, 11.57039), (48.17737, 11.56999), (48.17733, 11.56986), (48.17716, 11.56927), (48.17712, 11.56915), (48.17705, 11.56887), (48.17694, 11.5684), (48.17679, 11.56788), (48.17677, 11.56779), (48.17676, 11.56769), (48.17672, 11.56754), (48.17649, 11.5675), (48.17639, 11.56749), (48.17623, 11.56744), (48.17613, 11.56736), (48.17606, 11.56728), (48.17598, 11.56713), (48.17595, 11.56702), (48.17588, 11.56681), (48.17581, 11.56661), (48.17579, 11.56656), (48.17574, 11.56646), (48.17573, 11.56644), (48.17568, 11.56636), (48.1756, 11.56627), (48.17556, 11.56621), (48.17548, 11.56616), (48.17545, 11.56614), (48.1753, 11.56607), (48.17498, 11.56594), (48.1

In [119]:
engine = CO2Engine(CO2Engine.CO2Profile())
engine.AccumulateFuelConsumption(32.383104317653725, 4)
engine.AccumulateFuelConsumption(70000, 4)
engine.fuelAccumulated_l

26.218836361463993

In [147]:
def CalculateRollingResistance(tire_pressure, velocity, force_Load):
    alpha = 1.03399904
    beta = -0.41081927
    a = 0.05933157
    b = 9.85526e-05
    c = 3.72314e-07

    Z = force_Load
    p = tire_pressure
    v = velocity

    Frr = (Z**alpha) * (p**beta) * (a+(b*v)+(c*v*v))
    return Frr


In [146]:
# assuming, at peak load, especially climbing, turbo will run at maximum possible compression, leading to maximum torque axle output. with displacement at 12.8l at peak power RPM 1400 rpm

mass_vehicle = 32000 # in kilograms
mass_load = 10000 # in kilograms
accel_gravity = 9.8 # m/s^2
accel_vehicle = 1 # m/s^2
wheel_radius = 1
radius_tyre = 0.515 # 515mm Tyre radius: FMX84 
torque_engine_max = 2600 # Nm, ATO2612 gearbox output, D11 380 hp (279 kW)
load_nominal = 0.6
milage_per_liter = 8.4746 #kmpl, community average with 11.8km per 100km, assumed at 60% load (load_nominal)
mileage_modifer = 5 #the vehicle can go plus or minus modifer amount
displacement_d13 = 12.8 # dm^3 or litre, D13 variant
axle_ratio_highest = 4.11 #maximum torque conversion at rear for power delivery, RSS1370A
emission_co2 = 3.17 # g per g of diesel
diesel_l_to_g = 850.8 #g per l
num_wheels = 6
tire_pressure = 240 #kPa 

theta = 0 # degrees
distance = 32.383104317653725 #meters
mass_total = mass_vehicle + mass_load
force_engine = ((4) * mass_total / num_wheels * accel_gravity)/axle_ratio_highest
force_weight = mass_total * accel_gravity
force_angular = force_weight * math.sin(math.radians(theta))
force_engine_max = (torque_engine_max * axle_ratio_highest) / radius_tyre
load = force_angular / force_engine_max
if(theta == 0): load = force_engine / force_engine_max
load_offset = load - load_nominal
milage_compensated = (milage_per_liter - ((load_offset)* mileage_modifer))
fuel_consumed = ((distance * 0.001) / milage_compensated)
co2_consumption = emission_co2 * (fuel_consumed * diesel_l_to_g)
force_engine_max, force_engine, force_angular, load, co2_consumption

print("Distance Total Measured: {} km\n\
Approximated max engine force delivery: {} N\n\
Current engine force delivery: {} N\n\
Current force exerted on vehicle due to angle change: {} N\n\
Load Ratio: {}(-)\n\
Load Offset: {}(-)\n\
Compensated Mileage: {} km/l\n\
Fuel Consumed: {} ml\n\
Corresponding CO2 Emissions: {} kg\
".format(distance/1000, force_engine_max, force_engine, force_angular, load, load_offset, milage_compensated, fuel_consumed*1000, co2_consumption/1000))


Distance Total Measured: 0.032383104317653726 km
Approximated max engine force delivery: 20749.514563106797 N
Current engine force delivery: 66763.99026763989 N
Current force exerted on vehicle due to angle change: 0.0 N
Load Ratio: 3.217616974343491(-)
Load Offset: 2.6176169743434907(-)
Compensated Mileage: -4.6134848717174535 km/l
Fuel Consumed: -7.019228461368841 ml
Corresponding CO2 Emissions: -0.018931111852536372 kg


In [158]:
# assuming, at peak load, especially climbing, turbo will run at maximum possible compression, leading to maximum torque axle output. with displacement at 12.8l at peak power RPM 1400 rpm

mass_vehicle = 32000 # in kilograms
mass_load = 10000 # in kilograms
accel_gravity = 9.8 # m/s^2
accel_vehicle = 1 # m/s^2
wheel_radius = 1
radius_tyre = 0.515 # 515mm Tyre radius: FMX84 
torque_engine_max = 2600 # Nm, ATO2612 gearbox output, D11 380 hp (279 kW)
load_nominal = 0.5
milage_per_liter = 5.4746 #kmpl, community average with 11.8km per 100km, assumed at 60% load (load_nominal)
mileage_modifer = 5 #the vehicle can go plus or minus modifer amount
displacement_d13 = 12.8 # dm^3 or litre, D13 variant
axle_ratio_highest = 4.11 #maximum torque conversion at rear for power delivery, RSS1370A
emission_co2 = 3.17 # g per g of diesel
diesel_l_to_g = 850.8 #g per l
num_wheels = 6
tire_pressure = 240 #kPa 
rolling_pressure_ratio = 0.01 #https://x-engineer.org/rolling-resistance/


theta = -0 # degrees
distance = 10.32549612379126 #meters
mass_total = mass_vehicle + mass_load
force_engine = ((4) * mass_total / num_wheels * accel_gravity)/axle_ratio_highest
force_weight = mass_total * accel_gravity
force_angular = (mass_total * accel_gravity * math.sin(math.radians(theta))) + (rolling_pressure_ratio * mass_total * accel_gravity * math.cos(math.radians(theta)))
force_engine_max = (torque_engine_max * axle_ratio_highest) / radius_tyre
load = force_angular / force_engine_max
if(theta == 0): load = force_engine / force_engine_max
load_offset = load - load_nominal
milage_compensated = max(milage_per_liter - mileage_modifer, (milage_per_liter - ((load_offset)* mileage_modifer)))
fuel_consumed = ((distance * 0.001) / milage_compensated)
co2_consumption = emission_co2 * (fuel_consumed * diesel_l_to_g)
force_engine_max, force_engine, force_angular, load, co2_consumption

print("Distance Total Measured: {} km\n\
Approximated max engine force delivery: {} N\n\
Current engine force delivery: {} N\n\
Current force exerted on vehicle due to angle change: {} N\n\
Load Ratio: {}(-)\n\
Load Offset: {}(-)\n\
Compensated Mileage: {} km/l\n\
Fuel Consumed: {} ml\n\
Corresponding CO2 Emissions: {} kg\
".format(distance/1000, force_engine_max, force_engine, force_angular, load, load_offset, milage_compensated, fuel_consumed*1000, co2_consumption/1000))


Distance Total Measured: 0.010325496123791258 km
Approximated max engine force delivery: 20749.514563106797 N
Current engine force delivery: 66763.99026763989 N
Current force exerted on vehicle due to angle change: 4116.0 N
Load Ratio: 3.217616974343491(-)
Load Offset: 2.717616974343491(-)
Compensated Mileage: 0.4745999999999997 km/l
Fuel Consumed: 21.756207593323364 ml
Corresponding CO2 Emissions: 0.05867727510266647 kg
